In [1]:
import cv2
import numpy as np
import math
from PIL import Image

In [2]:
global img_3,top_left,top_right,bottom_left,bottom_right, isDrawing,isMoving, isRotating, isScaling, height, width
isDrawing = False
isMoving = False
isRotating = False
isScaling = False
img_3 = np.zeros([720,1080,3],dtype=np.uint8)
img_3.fill(255)
height, width = img_3.shape[:2]

In [3]:
def circleCorner(top_left, top_right, bottom_left, bottom_right):
    cv2.circle(img_3, top_left, 5, (0,255,0),-1)
    cv2.circle(img_3, top_right, 5, (0,0,255),-1)
    cv2.circle(img_3, bottom_left, 5, (255,0,0),-1)
    cv2.circle(img_3, bottom_right, 5, (0,0,0),-1)

In [4]:
def lineRect(top_left, top_right, bottom_left, bottom_right):
    cv2.line(img_3, top_left, top_right, (0,0,0), 2)
    cv2.line(img_3, top_right, bottom_right, (0,0,0), 2)
    cv2.line(img_3, bottom_right, bottom_left, (0,0,0), 2)
    cv2.line(img_3, bottom_left, top_left, (0,0,0), 2)

In [5]:

def drawRectangle(action, x, y, flags, *userdata):
  # Referencing global variables 
  global top_left_corner, bottom_right_corner, img_3, top_left,top_right,bottom_left,bottom_right,isDrawing
  # Mark the top left corner when left mouse button is pressed
  if action == cv2.EVENT_LBUTTONDOWN:
    top_left_corner = [x,y]
    isDrawing = True
  if action == cv2.EVENT_MOUSEMOVE and isDrawing == True:
    # Draw the rectangle
    bottom_right_corner = [x,y]
    img_3 = np.zeros([720,1080,3],dtype=np.uint8)
    img_3.fill(255)
    cv2.rectangle(img_3, top_left_corner, bottom_right_corner, (0,0,0),thickness=2)
    top_left = [top_left_corner[0], top_left_corner[1]]
    top_right = [top_left_corner[0], bottom_right_corner[1]]
    bottom_left = [bottom_right_corner[0], top_left_corner[1]]
    bottom_right = [bottom_right_corner[0], bottom_right_corner[1]]
    circleCorner(top_left, top_right, bottom_left, bottom_right)
    cv2.imshow("3 Channel Window",img_3)
    # When left mouse button is released, mark bottom right corner
  if action == cv2.EVENT_LBUTTONUP:
    print(top_left_corner)
    print(bottom_right_corner)
    isDrawing = False

In [6]:
def cal_move_rect(first_transform, bottom_transform, top_left,top_right,bottom_left,bottom_right, sx = 1, sy = 1):
    vector = [bottom_transform[0] - first_transform[0],bottom_transform[1]- first_transform[1]]
    tx=vector[0]
    ty=vector[1]
    T = np.array([[sx,0,tx],[0,sy,ty],[0,0,1]])
    top_left = np.array(top_left)
    top_right = np.array(top_right)
    bottom_left = np.array(bottom_left)
    bottom_right = np.array(bottom_right)
    rectangle = np.array([top_left,top_right,bottom_left,bottom_right])
    one = np.ones((rectangle.shape[0], 1))
    rectangle_matrix_ready = np.concatenate((rectangle,one), axis = 1)
    copyOfMatrixReady = rectangle_matrix_ready.copy()
    for i in range(rectangle_matrix_ready.shape[0]):
        copyOfMatrixReady[i] = T.dot(rectangle_matrix_ready[i])
    top_left = [int(copyOfMatrixReady[0][0]), int(copyOfMatrixReady[0][1])]
    top_right = [int(copyOfMatrixReady[1][0]), int(copyOfMatrixReady[1][1])]
    bottom_left = [int(copyOfMatrixReady[2][0]), int(copyOfMatrixReady[2][1])]
    bottom_right = [int(copyOfMatrixReady[3][0]), int(copyOfMatrixReady[3][1])]
    return top_left,top_right,bottom_left,bottom_right

In [7]:
def moveRectangle(action, x, y, flags, *userdata):
    global first_transform, bottom_transform, img_3, top_left,top_right,bottom_left,bottom_right,isMoving
    if action == cv2.EVENT_LBUTTONDOWN:
        first_transform = [x,y]
        isMoving = True
        # When left mouse button is released, mark bottom right corner
    if action == cv2.EVENT_MOUSEMOVE and isMoving == True:
        bottom_transform = [x,y]
        top_left,top_right,bottom_left,bottom_right = cal_move_rect(first_transform, bottom_transform, top_left,top_right,bottom_left,bottom_right)
        first_transform = bottom_transform
        img_3 = np.zeros([720,1080,3],dtype=np.uint8)
        img_3.fill(255)
        # print('new_top_left_corner',top_left_corner)
        # print('new_bottom_right_corner',bottom_right_corner)
        lineRect(top_left, top_right, bottom_left, bottom_right)
        circleCorner(top_left, top_right, bottom_left, bottom_right)
        cv2.imshow("3 Channel Window",img_3)

    if action == cv2.EVENT_LBUTTONUP:
        isMoving = False
        
        

In [8]:
def Calculate_degree(Center, Pointer):
    if Pointer[0] == Center[0]: # để tránh bị lỗi chia cho 0
        if Pointer[1] == Center[1]:
            return 180
        if Pointer[1] > Center[1]:
            return 90
        else:
            return 270
    if Pointer[1] == Center[1]:
        if Pointer[0] > Center[0]:
            return 0
        else:
            return 180
    degree = math.degrees(math.atan((Pointer[1] - Center[1])/(Pointer[0] - Center[0])))
    if degree < 0:
        degree += 360
    if Pointer[0] - Center[0] > 0: # x lon hon 0 
        if Pointer[1] - Center[1] > 0: # y lon hon 0 -> goc phan tu thu 1
            if degree > 90: 
                degree -= 180
        else: # goc phan tu thu 4
            if degree < 270:
                degree += 180
    else: # x nho hon 0
        if Pointer[1] - Center[1] > 0: # y lon hon 0 -> goc phan tu thu 2
            if degree > 180: 
                degree -= 180
        else: # goc phan tu thu 3
            if degree < 180:
                degree += 180 
    #degree = 360 - degree
    return degree

In [9]:
def rotateRectangle(action, x, y, flags, *userdata):
    global Center, Pointer, img_3, top_left,top_right,bottom_left,bottom_right, isRotating, new_top_left, new_top_right, new_bottom_left, new_bottom_right
    if action == cv2.EVENT_LBUTTONDOWN:
        Center = [x,y]
        isRotating = True
        new_top_left, new_top_right, new_bottom_left, new_bottom_right = top_left, top_right, bottom_left, bottom_right
    elif action == cv2.EVENT_MOUSEMOVE and isRotating == True:
        Pointer = [x,y]
        # top_left,top_right,bottom_left,bottom_right = cal_rotote_rect(Center, Pointer, top_left,top_right,bottom_left,bottom_right)
        new_top_left, new_top_right, new_bottom_left, new_bottom_right = cal_move_rect(Center, [0,0], top_left, top_right, bottom_left, bottom_right)
        degree = Calculate_degree(Center, Pointer)
        rectangle = np.array([new_top_left, new_top_right, new_bottom_left, new_bottom_right])
        theta = np.radians(degree)
        
        #(debug theta) theta dung?
        #print(theta)
        rectangle = rectangle.astype('float')
        rectangle_copy = rectangle.copy()
        r = np.array([[np.cos(theta),-np.sin(theta)],[np.sin(theta),np.cos(theta)]])
        for i in range(rectangle_copy.shape[0]):
            rectangle_copy[i] = r.dot(rectangle[i])
        new_top_left = rectangle_copy[0]
        new_top_right = rectangle_copy[1]
        new_bottom_left = rectangle_copy[2]
        new_bottom_right = rectangle_copy[3]
        new_top_left, new_top_right, new_bottom_left, new_bottom_right = cal_move_rect([0,0], Center, new_top_left, new_top_right, new_bottom_left, new_bottom_right)
        img_3 = np.zeros([720,1080,3],dtype=np.uint8)
        img_3.fill(255)
        cv2.line(img_3, Center, Pointer, (255,0,0), 2)
        cv2.line(img_3, Center, (Center[0]+ 90, Center[1]), (255,0,0), 2)
        cv2.putText(img_3,str(360 - degree), (Pointer[0],Pointer[1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 1)
        lineRect(new_top_left, new_top_right, new_bottom_left, new_bottom_right)
        circleCorner(new_top_left, new_top_right, new_bottom_left, new_bottom_right)
        cv2.imshow("3 Channel Window", img_3)
    if action == cv2.EVENT_LBUTTONUP:
        top_left, top_right, bottom_left, bottom_right = new_top_left, new_top_right, new_bottom_left, new_bottom_right
        isRotating = False
        

In [10]:
def scaleRectangles(action, x, y, flags, *userdata):
    global scaleHead, scaleTail, img_3, top_left,top_right,bottom_left,bottom_right, isScaling
    if action == cv2.EVENT_LBUTTONDOWN:
        scaleHead = [x,y]
        isScaling = True
    if action == cv2.EVENT_MOUSEMOVE and isScaling == True: 
        scaleTail = [x,y]
        if scaleTail[0] > scaleHead[0]:
            dist = (math.dist(list(scaleHead), list(scaleTail))/width)+ 1
        else:
            dist = 1- (math.dist(list(scaleHead), list(scaleTail))/width)
        top_left,top_right,bottom_left,bottom_right = cal_move_rect(scaleHead, scaleHead, top_left,top_right,bottom_left,bottom_right, dist, dist)
        scaleHead = scaleTail
        img_3 = np.zeros([720,1080,3],dtype=np.uint8)
        img_3.fill(255)
        lineRect(top_left, top_right, bottom_left, bottom_right)
        circleCorner(top_left, top_right, bottom_left, bottom_right)
        cv2.imshow("3 Channel Window", img_3)
    if action == cv2.EVENT_LBUTTONUP:
        isScaling = False

In [11]:
img_3 = np.zeros([720,1080,3],dtype=np.uint8)
img_3.fill(255)
# or img[:] = 255
# highgui function called when mouse events occur
cv2.namedWindow("3 Channel Window")
enable_draw_rectangle = False
enable_transform_rectangle = False
enable_rotate_rectangle = False
enable_scale_rectangle = False
while(1):
    cv2.imshow('3 Channel Window', img_3)
    k = cv2.waitKey(0)
    # if d is pressed, can draw
    if (k == 100): 
        enable_draw_rectangle = not enable_draw_rectangle
        enable_transform_rectangle = False
        enable_rotate_rectangle = False
        enable_scale_rectangle = False
        if enable_draw_rectangle:
            cv2.setMouseCallback("3 Channel Window", drawRectangle)
        else:
            cv2.setMouseCallback("3 Channel Window", lambda *args : None)
    
    # if t is pressed, can transform
    if (k == 116): 
        pass
        enable_transform_rectangle = not enable_transform_rectangle
        enable_draw_rectangle = False
        enable_rotate_rectangle = False
        enable_scale_rectangle = False
        if enable_transform_rectangle: 
            cv2.setMouseCallback("3 Channel Window", moveRectangle)
        else:
            cv2.setMouseCallback("3 Channel Window", lambda *args : None)

    # if r is pressed, can rotate
    if (k == 114):
        pass
        enable_rotate_rectangle = not enable_rotate_rectangle
        enable_draw_rectangle = False
        enable_transform_rectangle = False
        enable_scale_rectangle = False
        if enable_rotate_rectangle:
           cv2.setMouseCallback("3 Channel Window", rotateRectangle)
        else:
            cv2.setMouseCallback("3 Channel Window", lambda *args : None)

    # if s is pressed, can scale
    if (k == 115):
        pass
        enable_scale_rectangle = not enable_scale_rectangle
        enable_draw_rectangle = False
        enable_transform_rectangle = False
        enable_rotate_rectangle = False
        if enable_scale_rectangle:
            cv2.setMouseCallback("3 Channel Window", scaleRectangles)
        else:
            cv2.setMouseCallback("3 Channel Window", lambda *args : None)

    #if u is pressed, can input draw
    if (k == 117):
        pressone = True
        if pressone:
            pressone = False
            input_top_left_corner_x = input('input_top_left_corner_x:')
            input_top_left_corner_y = input('input_top_left_corner_y:')
            input_bottom_right_corner_x = input('input_bottom_right_corner_x:')
            input_bottom_right_corner_y = input('input_bottom_right_corner_y:')
            top_left_corner = [int(input_top_left_corner_x), int(input_top_left_corner_y)]
            bottom_right_corner = [int(input_bottom_right_corner_x), int(input_bottom_right_corner_y)]
            img_3 = np.zeros([720,1080,3],dtype=np.uint8)
            img_3.fill(255)
            cv2.rectangle(img_3, top_left_corner, bottom_right_corner, (0,0,0),thickness=2)
            top_left = [top_left_corner[0], top_left_corner[1]]
            top_right = [top_left_corner[0], bottom_right_corner[1]]
            bottom_left = [bottom_right_corner[0], top_left_corner[1]]
            bottom_right = [bottom_right_corner[0], bottom_right_corner[1]]
            circleCorner(top_left, top_right, bottom_left, bottom_right)
            cv2.imshow("3 Channel Window",img_3)
        pass

    #if i is pressed, can input transform
    if (k == 105):
        pressone = True
        if pressone:
            pressone = False
            input_lech_phai_bao_nhieu = int(input('input_lech_phai_bao_nhieu:'))
            input_xuong_duoi_bao_nhieu = int(input('input_xuong_duoi_bao_nhieu:'))
            top_left,top_right,bottom_left,bottom_right = cal_move_rect([0,0], [input_lech_phai_bao_nhieu, input_xuong_duoi_bao_nhieu], top_left,top_right,bottom_left,bottom_right)
            img_3 = np.zeros([720,1080,3],dtype=np.uint8)
            img_3.fill(255)
            lineRect(top_left, top_right, bottom_left, bottom_right)
            circleCorner(top_left, top_right, bottom_left, bottom_right)
            cv2.imshow("3 Channel Window",img_3)
    
    #if o is pressed, can input rotate
    if (k == 111):
        pressone = True
        if pressone:
            pressone = False
            center = [(top_left[0] + bottom_right[0])/2, (top_left[1] + bottom_right[1])/2]
            input_xoay_bao_nhieu = int(input('input_xoay_bao_nhieu:'))
            input_xoay_bao_nhieu = 360 - input_xoay_bao_nhieu
            top_left, top_right, bottom_left, bottom_right = cal_move_rect(center, [0,0], top_left, top_right, bottom_left, bottom_right)
            rectangle = np.array([top_left, top_right, bottom_left, bottom_right])
            theta = np.radians(input_xoay_bao_nhieu)  
                      
            rectangle = rectangle.astype('float')
            rectangle_copy = rectangle.copy()
            r = np.array([[np.cos(theta),-np.sin(theta)],[np.sin(theta),np.cos(theta)]])
            for i in range(rectangle_copy.shape[0]):
                rectangle_copy[i] = r.dot(rectangle[i])
            top_left = rectangle_copy[0]
            top_right = rectangle_copy[1]
            bottom_left = rectangle_copy[2]
            bottom_right = rectangle_copy[3]
            top_left, top_right, bottom_left, bottom_right = cal_move_rect([0,0], center, top_left, top_right, bottom_left, bottom_right)
            img_3 = np.zeros([720,1080,3],dtype=np.uint8)
            img_3.fill(255)
            lineRect(top_left, top_right, bottom_left, bottom_right)
            circleCorner(top_left, top_right, bottom_left, bottom_right)
            cv2.imshow("3 Channel Window",img_3)
    #if p is pressed, can input scale

    if (k == 112):
        pressone = True
        if pressone:
            pressone = False
            center = [(top_left[0] + bottom_right[0])/2, (top_left[1] + bottom_right[1])/2]
            input_scale_bao_nhieu = float(input('input_scale_bao_nhieu:'))
            top_left,top_right,bottom_left,bottom_right = cal_move_rect(center, center, top_left,top_right,bottom_left,bottom_right, input_scale_bao_nhieu, input_scale_bao_nhieu)
            img_3 = np.zeros([720,1080,3],dtype=np.uint8)
            img_3.fill(255)
            lineRect(top_left, top_right, bottom_left, bottom_right)
            circleCorner(top_left, top_right, bottom_left, bottom_right)
            cv2.imshow("3 Channel Window",img_3)
    
    # If c is pressed, out the image
    if (k == 99):
        img_3 = np.zeros([720,1080,3],dtype=np.uint8)
        img_3.fill(255)
        cv2.imshow('3 Channel Window', img_3)
    
    # If q is pressed, quit the program
    if (k == 113):
        break
cv2.destroyAllWindows()